![encodelogo](images/encodelogo.gif)

# Exploring ENCODE data from EC2 with Jupyter notebook

This notebook demonstrates how to mount ***s3://encode-public*** on an **EC2 instance** using **Goofys**. Once the bucket is mounted we can launch a **Jupyter notebook** on the instance and connect to it remotely. **Goofys** makes an **S3 bucket** appear as a normal file system, and is useful for tools that expect a local file path. The benefit of using **EC2** is that the compute is scalable to the analysis you would like to perform, and you don't have to download anything locally.

# Spin up instance

# SSH into the instance

Open up a terminal window and type:

```
$ ssh -i <~/.ssh/your-secret.pem> ubuntu@<your-ec2-instance-dns>
```

Make sure to fill in the path to your SSH key and the public DNS of your **ECS instance**.

# Install dependencies

# Mount S3 bucket

**Goofys** expects valid AWS credentials (though they don't need to have permission to do anything since we are mounting a public bucket). 

* Run `aws configure` and enter *aws_secret_access_key/aws_access_key_id*
* Run `cat ~/.aws/credentials` to confirm credentials are stored

Now we can use **Goofys** to mount the **S3 bucket** to a folder (also called *encode-public*) on our **EC2 instance**:

```bash
$ $GOPATH/bin/goofys encode-public/ encode-public/
```

# Start Jupyter notebook

```
$ jupyter notebook --no-browser --port=8888
```

*Make a note of the token that is returned so we can connect to the running Jupyter server remotely.*

# Link local port to remote port

Open another terminal window and type:

```
ssh -i ~/.ssh/<your-secret.pem> -L 8001:localhost:8888 ubuntu@<your-ec2-instance-dns>
```

This links your local 8001 port to the Jupyter notebook running on port 8888 of your EC2 instance. Launch a browser and type in `localhost:8001`. You should see a Jupyter window asking you for the token from above.

# Create notebook

# Open ENCODE file using local path

Use tree